In [9]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback

In [6]:
load_dotenv()
KEY=os.getenv("OPENAI_API_KEY")

In [8]:
llm=ChatOpenAI(openai_api_key=KEY, model_name="gpt-4o", temperature=0.5)

C:\Users\Suraj\AppData\Local\Temp\ipykernel_21876\2360619983.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(openai_api_key=KEY, model_name="gpt-4o", temperature=0.5)


In [12]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [13]:
TEMPLATE="""
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number}MCQs
### RESPONSE_JSON
{response_json}
"""

In [14]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [15]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key='quiz',verbose='True')

C:\Users\Suraj\AppData\Local\Temp\ipykernel_21876\4044163263.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key='quiz',verbose='True')


In [16]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [17]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["quiz", "subject"],
    template=TEMPLATE2
)

In [18]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key='review',verbose='True')

In [19]:
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone","response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)

In [22]:
with open(r"D:\Data\Docs PKJHA 29-12-24\B. TECH\Udemy\Python Programming\Generative AI\Project\Project 1\data.txt", "r") as f:
    text = f.read()
print(text)

Generative artificial intelligence (generative AI, GenAI,[1] or GAI) is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2][3][4] These models learn the underlying patterns and structures of their training data and use them to produce new data[5][6] based on the input, which often comes in the form of natural language prompts.[7][8]

Improvements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini, and LLaMA; text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video AI generators such as Sora.[9][10][11][12] Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed generative AI models.[7][13][14]

Generative AI has uses across a wide r

In [23]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [52]:
#How to setup Token usage tracking in Langchain
#https://python.langchain.com/docs/get_started/introduction/usage_tracking

with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": text,
            "number": 3,
            "subject": "Gen AI",
            "tone": "Normal",
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )
    



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Generative artificial intelligence (generative AI, GenAI,[1] or GAI) is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2][3][4] These models learn the underlying patterns and structures of their training data and use them to produce new data[5][6] based on the input, which often comes in the form of natural language prompts.[7][8]

Improvements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini, and LLaMA; text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video AI generators such as Sora.[9][10][11][12] Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as n

In [53]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): {cb.total_cost}")

Total Tokens: 1492
Prompt Tokens: 940
Completion Tokens: 552
Total Cost (USD): 0.00787


In [54]:
response

{'text': 'Generative artificial intelligence (generative AI, GenAI,[1] or GAI) is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2][3][4] These models learn the underlying patterns and structures of their training data and use them to produce new data[5][6] based on the input, which often comes in the form of natural language prompts.[7][8]\n\nImprovements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini, and LLaMA; text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video AI generators such as Sora.[9][10][11][12] Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed generative AI models.[7][13][14]\n\nGenerative AI has uses a

In [55]:
response.get('quiz')

'```json\n{\n  "1": {\n    "mcq": "What is generative artificial intelligence primarily used for?",\n    "options": {\n      "a": "Producing new data based on learned patterns",\n      "b": "Improving existing machine learning algorithms",\n      "c": "Analyzing large datasets for insights",\n      "d": "Automating repetitive tasks"\n    },\n    "correct": "a"\n  },\n  "2": {\n    "mcq": "Which of the following is NOT a generative AI model mentioned in the text?",\n    "options": {\n      "a": "ChatGPT",\n      "b": "Stable Diffusion",\n      "c": "TensorFlow",\n      "d": "DALL-E"\n    },\n    "correct": "c"\n  },\n  "3": {\n    "mcq": "Which industry is NOT mentioned as a user of generative AI in the text?",\n    "options": {\n      "a": "Healthcare",\n      "b": "Finance",\n      "c": "Automobile manufacturing",\n      "d": "Entertainment"\n    },\n    "correct": "c"\n  }\n}\n```'

In [62]:
quiz=response.get("quiz").strip("```json\n")

In [64]:
quiz=json.loads(quiz)

In [65]:
quiz_table_data=[]
for key,value in quiz.items():
    mcq=value.get("mcq")
    options=" | ".join(
        [
            f"{option}:{option_value}"
            for option,option_value in value["options"].items()
        ]
    )
    correct=value.get("correct")
    quiz_table_data.append(
        {
            "MCQ":mcq,
            "Choices":options,
            "Correct":correct
        }
    )

In [66]:
quiz_table_data

[{'MCQ': 'What is generative artificial intelligence primarily used for?',
  'Choices': 'a:Producing new data based on learned patterns | b:Improving existing machine learning algorithms | c:Analyzing large datasets for insights | d:Automating repetitive tasks',
  'Correct': 'a'},
 {'MCQ': 'Which of the following is NOT a generative AI model mentioned in the text?',
  'Choices': 'a:ChatGPT | b:Stable Diffusion | c:TensorFlow | d:DALL-E',
  'Correct': 'c'},
 {'MCQ': 'Which industry is NOT mentioned as a user of generative AI in the text?',
  'Choices': 'a:Healthcare | b:Finance | c:Automobile manufacturing | d:Entertainment',
  'Correct': 'c'}]

In [68]:
quiz=pd.DataFrame(quiz_table_data)

In [69]:
quiz.to_csv("Gen AI.csv",index=False)